# Trabajo práctico 1 : Analisis exploratorio del dataset Properatti

Grupo #11: Camila Coltriani, Irania Fuentes, Johnatan Fischelson, Luis Dartayet, Ornela Cevolli  

## Introducción: 
El dataset Properatti está construido con los datos de venta de propiedades en diferentes provincias de Argentina; incluye ubicacion política y georeferenciada, así como los precios, superficie, cantidad de habitaciones y pisos, expensas y, otras informaciones. En este dataset cada fila es una propiedad en venta.

## Identificar el problema

El objetivo de este trabajo es realizar una limpieza del dataset properatti con la finalidad de obtener un dataset final con datos confiables que pueda ser utilizado en la generación de un modelo estadistico posterior.
Con base en esto se plantean los siguientes objetivos especificos:
 - Adquirir los datos: leer y conocer su estructura para determinar las herramientas apropiadas para su manipulación.

 - Parsear los datos: realizar el analisis exploratorio de los datos que permita verificar la existencia o no de relaciones entre variables, valores duplicados, valores faltantes, valores atípicos o valores erroneos que para validar o aumentar la confiabilidad de los datos.

 - Minar los datos: aplicar las herramientas de python para corregir datos erroneos o duplicados, completar/eliminar valores nulos.
 
 - Refinar los datos: eliminar variables redundantes o repetidas, crear nuevas variables y dar un formato limpio al dataset original.

## Adquirir los datos

In [ ]:
#Importando las bibliotecas necesarias para trabajar el dataset properatti
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
#import geopandas as gpd

TODO: otros recursos utilizados
- archivo de id_geonames: ar_copy.csv
- archivo de barrios Argentina: barrios.csv 

In [ ]:
# Leemos y cargamos el dataset properatti.csv en una variable 
data = pd.read_csv("./properatti.csv", index_col=0)

In [ ]:
# Visualización de la forma y atributos del dataset 
print(data.shape)
print("El dataset está compuesto por:", data.shape[0], "filas y",data.shape[1],"columnas.")
data.sample(5) #

## Parsear los datos

### Analisis exploratorio general del dataset de Properatti

#### Descripción de las columnas del dataset:

Los atributos o columas que incluye son:

● unmaded: 0: indice de filas

● property_type: tipo de inmueble en venta (casa, departamento, ph...)

● operation: tipo de operacion inmobiliaria para las propiedades 

● place_name: ubicacion del inmueble por ciudad/Partido o barrios

● place_with_parent_names: ubicacion agrupada del inmueble (Pais|Provincia|Partido o barrio)

● country_name: nombre del país donde ocurre la operacion inmobiliaría

● state_name: ubicacion del inmueble por provincia

● geonames_id: número de identificación en la base de datos GeoNames asociado a la ubicacion por coordenadas

● lat-lon: ubicacion de latitud y longitud concatenada

● lat  ●lon: ubicacion de latitud y longitud en columnas separadas

● price: precio del inmueble

● currency: divisa en la que está expresado el precio del inmueble

● price_aprox_local_currency: Precio aproximado en la moneda local del país de publicación

● surface_total_in_m2: superficie total m² del inmueble

● surface_covered_in_m2: Superficie cubierta en m²

● price_usd_per_m2: Precio en dolares por metro cuadrado (USD/m²: precio dólares / superficie)

● price_per_m2: Precio del metro cuadrado del inmueble

● floor: N° de piso (cuando corresponde)

● room: cantidad de habitaciones

● expenses: expensas (cuando corresponde)

● properati_url	: URL de la inmobiliaría Properati en la Web

● description: descripción del inmueble en la publicación Web

● title: título del inmueble en la publicación

● image_thumbnail: URL de un thumbnail de la primer foto en la Web

In [ ]:
#Identificamos el tipo de dato de cada columna
data.dtypes
# El tipo de datos para variables cuantitativas discreta como floor y rooms deberia ser int, 
# posiblemente tengamos que realizar el cambio en su manipulación.

In [ ]:
#Realizamos una descripcion de todas las columnas para ver el n° de registros unicos por columna, el dato más frecuente y 
# su cantidad, tanto para las columnas cualitativas como numericas
data.describe(include="all")

# Algunas interpretaciones/inferencias:
# operation y country_name tiene 1 solo dato:  Sell y Argentina, como ya sabiamos, el dataset son datos de venta en Argentina
# Existen cuatro tipos de propiedades en venta, la más frecuente es apartamento
# Placename tiene como dato más frecuente la ciudad de Cordoba y state_name tiene a Capital Federal
# lat-long hay datos repetidos o son los mismos edificios representados en un area determinada
# Existen valores maximos muy alejados del resto de datos en las columnas de superfice, floor y rooms, posibles outliers

In [ ]:
#Identificamos los valores unicos x columna
for columnas in data.columns:
    print("")
    print(f'Nombre:{columnas}')
    print(data[columnas].value_counts())

#de esta funcion sumamos información general sobre el data set:
#  identificamos los tipos de inmueble en venta: apartamentos y casas concentran la mayoria de datos
#  las divisas más utilizadas son el peso argentina y dolares, hay datos que podemos tomar como no representativos:
#  el PEN: peso peruano y UYU: peso uruguayo ya que no pasan de dos registros en el dataset. 
# para floor y rooms hay que tratar los valores outliers

In [ ]:
#Identificamos los registros nan por columna

print(data.isna().sum())
#price, currency, price_aprox, price_usd tienen la misma cantidad de nulos 20410
#hay que averiguar si price_per_m2 es la relacion de price y surface_total, así podriamos completar nulos en price_per_m2

In [ ]:
# Vemos la relacion de registros nan con respecto al total de registros
data.isna().sum()/data.shape[0] *100

#podriamos completar place_name por el %bajo de nan
#el mayor % de nan se encuentra en floor, rooms y expensas

In [ ]:
# Identificamos si existe una correlacion entre variables

corr = data.set_index('place_name').corr()
sm.graphics.plot_corr(corr, xnames=list(corr.columns))
plt.show()

#Hay una correlacion entre price y price_aprox_local_currency: price está en dolares/pesos argentinos y local currency en pesos, podría 
#deberse al tipo de cambio utilizado.

Para seguir con el analisis exploratorio de los datos, hemos decidido dividir el dataset en dos grandes grupos por su relación: 
- columnas relacionadas a la ubicación política y geografica: property_type, place_name, place_with_parent_names,	country_name, state_name, geonames_id
- columnas con datos numericos que agrupan para las propiedades, el precio, el precio por superficie y datos fisicos del inmueble como el numero de pisos, habitaciones. 
- y las columnas de descripcion del inmueble asocida a las columnas anteriores por contener informacion sobre superficie y division del inmueble 

TODO: MOVER AL ANALISIS DE DATOS DE GEOUBICACION
Analizamos por lat-lon para si había duplicados considerando que las mismas coordenadas son las misma propiedad.

No es posible determinar si es la misma propiedad porque lat-lon se refiere muchas veces a la ubicación aproximada. 


In [ ]:
## ver datos duplicados
data_copy = data.copy()
data_copy.dropna(subset=['lat-lon'], inplace=True)
data_copy_group = data_copy.groupby('lat-lon').count()
data_copy_group[data_copy_group['operation'] > 1].sort_values(by='operation', ascending=False)

In [ ]:
## ver si existe algun dato duplicado
data.duplicated().any()

In [ ]:
data['lat-lon'].duplicated().any()

In [ ]:
data_copy.shape()

In [ ]:
data_copy[data_copy['lat-lon'] == '-34.4026444,-58.6684776']

#### Análisis de datos faltantes

In [ ]:
print(data.isna().sum())

In [ ]:
data.isna().sum()/data.shape[0] *100

In [ ]:
missing_data = data.isna().sum(axis=0)
missing_data_df = pd.DataFrame(missing_data, columns=['count'])
missing_data_df['perc'] = (missing_data_df / data.shape[0]).round(2)*100
missing_data_df.sort_values(ascending=False, by='count')

#### Dispersión de datos

In [ ]:
data_dispersion = data.apply(lambda x: x.unique().size)
data_dispersion_df = pd.DataFrame(data_dispersion, columns=['count'])
data_dispersion_df["perc"] = (data_dispersion / data.shape[0]).round(2)*100
data_dispersion_df.sort_values(ascending=True, by='count')

In [ ]:
for col in data.columns:
    if(data[col].nunique() < 100):
        print(col)
        print(data[col].unique())
        print()

#### Analisis de correlacion entre columnas

### Verificar la calidad de los datos:
- ver consistencia de los datos: duplicación de columnas 


## Minar los datos

## Refinar los datos

## Exportar el nuevo dataset 